# API Asos

In [15]:
import json
import requests
import pandas as pd
import os
import sys
import dotenv

dotenv.load_dotenv()

sys.path.append(os.path.abspath('../'))
from src import funciones_auxiliares as fa

Buscamos en la API de Asos vestidos de Mango, ya que es una de las marcas favoritas de las clientas:

In [ ]:
url = "https://asos10.p.rapidapi.com/api/v1/getProductList"
id_mango = 8264
api_key = os.getenv("api_asos")

querystring = {"categoryId":id_mango, "sort":"recommended", "limit":2000}

headers = {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "asos10.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
response_mango = response.json()

# Guardamos el resultado en datos
with open("../datos/api_asos/vestidos_mango.json", 'w') as file:
    json.dump(response_mango, file, indent=4)


Filtramos los productos que sean vestidos:

In [3]:
lista_id_vestido = []
dic_precio = {}
with open(f"../datos/api_asos/vestidos_mango.json", 'r') as file:
    response_mango = json.load(file)

for i in range(len(response_mango["data"]["products"])):
    if "dress" in response_mango["data"]["products"][i]["name"]:
        id = response_mango["data"]["products"][i]["id"]
        lista_id_vestido.append(id)
        dic_precio[id]=response_mango["data"]["products"][i]["price"]["current"]["value"]

In [39]:
len(lista_id_vestido)

47

Para cada vestido volvemos a llamar a la api para que nos de aún más información de cada producto.

In [ ]:
api_key = os.getenv("api_asos")

for id in lista_id_vestido:

    url = "https://asos10.p.rapidapi.com/api/v1/getProductDetails"

    querystring = {"productId":[id]}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "asos10.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    res = response.json()
    with open(f"../datos/api_asos/vestido_{id}.json", 'w') as file:
        json.dump(res, file, indent=4)

Generamos un diccionario limpio con todos los valores necesarios y creamos un daraframe.

In [9]:
dic_vestidos = fa.creacion_diccionario_asos(dic_precio)
df_mango = pd.DataFrame(dic_vestidos)

In [10]:
df_mango["color"] = df_mango["color"].map(fa.colores)

In [14]:
df_mango

,nombre,marca,precio,color,talla,disponible
0,Vestido midi plateado con cuello halter de len...,Mango,133.00,plata,S,True
1,Vestido midi plateado con cuello halter de len...,Mango,133.00,plata,M,True
2,Vestido midi plateado con cuello halter de len...,Mango,133.00,plata,L,True
3,Vestido midi negro con estampado floral y aber...,Mango,129.99,negro,XS,True
4,Vestido midi negro con estampado floral y aber...,Mango,129.99,negro,S,True
...,...,...,...,...,...,...
206,Vestido largo marrón oscuro sin mangas con pes...,Mango,99.99,marrón oscuro,XS,True
207,Vestido largo marrón oscuro sin mangas con pes...,Mango,99.99,marrón oscuro,S,True
208,Vestido largo marrón oscuro sin mangas con pes...,Mango,99.99,marrón oscuro,M,True
209,Vestido largo marrón oscuro sin mangas con pes...,Mango,99.99,marrón oscuro,L,True
